In [ ]:
!pip install gym[atari]

In [1]:
%matplotlib widget

In [1]:
from IPython.core.debugger import set_trace

In [2]:
import torch
from torch import nn
from torch.distributions import Categorical
from torch.optim import Adam, SGD

In [3]:
import numpy

In [4]:
from matplotlib import pyplot as plot

In [5]:
import copy
from time import sleep

In [6]:
import gym

In [7]:
from utils import Buffer, collect_one_episode, normalize_obs, copy_params, avg_params

In [8]:
device='cpu'

In [9]:
class ResLinear(nn.Module):
    def __init__(self, n_in, n_out, act=nn.ReLU()):
        super(ResLinear, self).__init__()
        self.act = act
        self.linear = nn.Linear(n_in, n_out)
        self.bn = nn.BatchNorm1d(n_out)
        
        assert(n_in == n_out)
    
    def forward(self, x):
        h = self.act(self.bn(self.linear(x)))
        return h + x

In [10]:
class Player(nn.Module):
    def __init__(self, n_in=128, n_hid=100, n_out=6):
        super(Player, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, n_out))
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, obs, normalized=False):
        if normalized:
            return self.softmax(self.layers(obs))
        else:
            return self.layers(obs)

In [11]:
class Qnet(nn.Module):
    def __init__(self, n_in=128, n_act=6, n_hid=100):
        super(Qnet, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid), 
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, n_act))
    
    def forward(self, obs, act):
#         set_trace()
        return self.layers(obs).gather(1, act.long())
    
    def value(self, obs):
        return self.layers(obs).max(dim=1)
    
    def q(self, obs):
        return self.layers(obs)

In [12]:
class Value(nn.Module):
    def __init__(self, n_in=128, n_hid=100):
        super(Value, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, 1))
    
    def forward(self, obs):
        return self.layers(obs)

In [13]:
env = gym.make('Pong-ram-v0')

n_frames = 4

In [14]:
# create a policy
player = Player(n_in=128*n_frames, n_hid=128, n_out=6).to(device)

In [15]:
# create a q estimator
qnet = Qnet(n_in=128*n_frames, n_hid=128, n_act=6).to(device)
qold = Qnet(n_in=128*n_frames, n_hid=128, n_act=6).to(device)
copy_params(qnet, qold)

In [16]:
# create a value estimator
value = Value(n_in=128*n_frames, n_hid=128).to(device)

In [17]:
# initialize optimizers
opt_player = Adam(player.parameters(), lr=0.0001)
opt_q = Adam(qnet.parameters(), lr=0.0001)
opt_value = Adam(value.parameters(), lr=0.0001)

In [18]:
# initialize replay buffer
replay_buffer = Buffer(max_items=50000, n_frames=n_frames)

In [19]:
n_iter = 1000
init_collect = 1
n_collect = 1
n_q = 100
n_value = 0
n_policy = 100
disp_iter = 1
val_iter = 1

max_len = 1000
batch_size = 1000

ent_coeff = 0. #0.001
discount_factor = .95

q_loss = -numpy.Inf
value_loss = -numpy.Inf
ret = -numpy.Inf
entropy = -numpy.Inf
valid_ret = -numpy.Inf

return_history = []

for ni in range(n_iter):
    player.eval()

    if numpy.mod(ni, val_iter) == 0:
        _, _, _, _, _, ret_ = collect_one_episode(env, player, max_len=max_len, deterministic=True, n_frames=n_frames)
        return_history.append(ret_)
        if valid_ret == -numpy.Inf:
            valid_ret = ret_
        else:
            valid_ret = 0.9 * valid_ret + 0.1 * ret_
        print('Valid run', ret_, valid_ret)

    # collect some episodes using the current policy
    # and push (obs,a,r,p(a)) tuples to the replay buffer.
    nc = n_collect
    if ni == 0:
        nc = init_collect
    for ci in range(nc):
        o_, r_, c_, a_, ap_, ret_ = collect_one_episode(env, player, max_len=max_len, discount_factor=discount_factor, n_frames=n_frames)
        replay_buffer.add(o_, r_, c_, a_, ap_)
        if ret == -numpy.Inf:
            ret = ret_
        else:
            ret = 0.9 * ret + 0.1 * ret_
    
    player.train()
    
    # fit a q function
    # TD learning: min_Q (Q(s,a) - (r + \gamma \max_a' Q(s',a')))^2
    for qi in range(n_q):
        opt_player.zero_grad()
        opt_value.zero_grad()
        opt_q.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_xn = torch.from_numpy(numpy.stack([ex['next']['obs'] for ex in batch]).astype('float32')).to(device)
        
        batch_y = torch.from_numpy(numpy.stack([ex['current']['rew'] for ex in batch]).astype('float32')).to(device)
        
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        
        q_pred = qnet(batch_x, batch_a).squeeze()
        q_next = qold.value(batch_xn)[0].squeeze()
        
        loss_ = ((q_pred - (batch_y + discount_factor * q_next)) ** 2)

        # since Q learning is off-policy, no need for importance weighting.
        iw = 1.
        loss = iw * loss_
        
        loss = loss.mean()
        
        loss.backward()
        opt_q.step()
        
    if q_loss == -numpy.Inf:
        q_loss = loss_.mean().item()
    else:
        q_loss = 0.9 * q_loss + 0.1 * loss_.mean().item()
        
    copy_params(qnet, qold)
        
    # fit a value function
    # a usual value estimator: min_V (V(s) - Q(s,a))^2
    for vi in range(n_value):
        opt_player.zero_grad()
        opt_value.zero_grad()
        opt_q.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)
        
        pred_y = value(batch_x).squeeze()
        pred_q = qnet(batch_x, batch_a).squeeze()
        
        loss_ = ((pred_y - pred_q) ** 2)
        
        batch_pi = player(batch_x, normalized=True)
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)
        logp = torch.log(batch_pi.gather(1, batch_a.long()))

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss_
        
        loss = loss.mean()
        
        loss.backward()
        opt_value.step()
        
    if value_loss < 0.:
        value_loss = loss_.mean().item()
    else:
        value_loss = 0.9 * value_loss + 0.1 * loss_.mean().item()
    
    if numpy.mod(ni, disp_iter) == 0:
        print('# plays', (ni+1) * n_collect, 'return', ret, 'value_loss', value_loss, 'q_loss', q_loss, 'entropy', -entropy)
    
    # fit a policy
    # advantage: (Q(a,s) - V(s))
    for pi in range(n_policy):
        opt_player.zero_grad()
        opt_value.zero_grad()
        opt_q.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex['current']['obs'] for ex in batch]).astype('float32')).to(device)
        batch_a = torch.from_numpy(numpy.stack([ex['current']['act'] for ex in batch]).astype('float32')[:,None]).to(device)

        batch_pi = player(batch_x, normalized=True)        
        logp = torch.log(batch_pi.gather(1, batch_a.long()))

        batch_q = qnet(batch_x, batch_a)
        batch_v = value(batch_x)
        
        # advantage
        adv = batch_q.clone().detach() - batch_v
        adv = adv / adv.abs().max()
        
        loss = -(adv * logp)
        
        batch_q = torch.from_numpy(numpy.stack([ex['current']['prob'] for ex in batch]).astype('float32')).to(device)

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss
        
        # entropy regularization: though, it doesn't look necessary in this specific case.
        ent = (batch_pi * torch.log(batch_pi)).sum(1)
        
        if entropy == -numpy.Inf:
            entropy = ent.mean().item()
        else:
            entropy = 0.9 * entropy + 0.1 * ent.mean().item()
        
        loss = (loss + ent_coeff * ent).mean()
        
        loss.backward()
        opt_player.step()

Valid run -20.0 -20.0
0 6
1 6
2 6
3 6
4 5
5 4
6 4
7 4
8 4
9 4
10 4
11 4
12 4
13 4
14 4
15 4
16 4
17 4
18 4
19 4
20 4
21 4
22 4
23 4
24 4
25 4
26 4
27 4
28 4
29 4
30 4
31 4
32 4
33 4
34 4
35 4
36 4
37 4
38 4
39 4
40 4
41 4
42 4
43 4
44 4
45 4
46 4
47 4
48 4
49 4
50 4
51 4
52 4
53 4
54 4
55 4
56 4
57 4
58 4
59 4
60 4
61 4
62 4
63 4
64 4
65 4
66 4
67 4
68 4
69 4
70 4
71 4
72 4
73 4
74 4
75 4
76 4
77 4
78 4
79 4
80 4
81 4
82 4
83 4
84 4
85 4
86 4
87 4
88 4
89 4
90 4
91 4
92 4
93 4
94 4
95 4
96 4
97 4
98 4
99 4
100 4
101 4
102 4
103 4
104 4
105 4
106 4
107 4
108 4
109 4
110 4
111 4
112 4
113 4
114 4
115 4
116 4
117 4
118 4
119 4
120 4
121 4
122 4
123 4
124 4
125 4
126 4
127 4
128 4
129 4
130 4
131 4
132 4
133 4
134 4
135 4
136 4
137 4
138 4
139 4
140 4
141 4
142 4
143 4
144 4
145 4
146 4
147 4
148 4
149 4
150 4
151 4
152 4
153 4
154 4
155 4
156 4
157 4
158 4
159 4
160 4
161 4
162 4
163 4
164 4
165 4
166 4
167 4
168 4
169 4
170 4
171 4
172 4
173 4
174 4
175 4
176 4
177 4
178 4
179 4
180 4
18

ValueError: all input arrays must have the same shape

In [ ]:
plot.figure()

plot.plot(return_history)
plot.grid(True)
plot.xlabel('# of plays x {}'.format(n_collect))
plot.ylabel('Return over the episode of length {}'.format(max_len))

plot.show()
plot.savefig('return_log.pdf', dpi=150)

In [22]:
# let the final policy play the pong longer
player.eval()
_, _, _, _, ret_ = collect_one_episode(env, player, max_len=1000000, deterministic=True, rendering=True)

KeyboardInterrupt: 